In [1]:
from transformers import set_seed
set_seed(2025)

In [2]:
import os
import gc
import time
import warnings

import pandas as pd
import polars as pl

import torch
import kaggle_evaluation.aimo_2_inference_server

pd.set_option('display.max_colwidth', None)
cutoff_time = time.time() + (4 * 60 + 45) * 60

# Load Model

In [3]:
from vllm import LLM, SamplingParams

warnings.simplefilter('ignore')

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

def clean_memory(deep=False):
    gc.collect()
    if deep:
        ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()

llm_model_pth = '/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-32b-awq/1'

llm = LLM(
    llm_model_pth,
    # dtype="half",                -> Changed this
    max_num_seqs=16,   #           -> Changed this
    max_model_len=8192, #32768,#4096*10,         
    trust_remote_code=True,     
    tensor_parallel_size=4,      
    gpu_memory_utilization=0.95, 
)

2025-02-06 22:02:48,314	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 02-06 22:03:14 awq_marlin.py:97] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
INFO 02-06 22:03:14 config.py:905] Defaulting to use mp for distributed inference
INFO 02-06 22:03:14 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-32b-awq/1', speculative_config=None, tokenizer='/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-32b-awq/1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


(VllmWorkerProcess pid=357) INFO 02-06 22:05:35 model_runner.py:1067] Loading model weights took 4.5671 GB
INFO 02-06 22:05:36 model_runner.py:1067] Loading model weights took 4.5671 GB
(VllmWorkerProcess pid=356) INFO 02-06 22:05:36 model_runner.py:1067] Loading model weights took 4.5671 GB
(VllmWorkerProcess pid=355) INFO 02-06 22:05:36 model_runner.py:1067] Loading model weights took 4.5671 GB
INFO 02-06 22:05:42 distributed_gpu_executor.py:57] # GPU blocks: 16008, # CPU blocks: 4096
INFO 02-06 22:05:42 distributed_gpu_executor.py:61] Maximum concurrency for 8192 tokens per request: 31.27x
INFO 02-06 22:05:45 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-06 22:05:45 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_m

In [4]:
tokenizer = llm.get_tokenizer()

# Utlities

#### Previous prompts used with this model from previous authors

In [5]:
# thoughts = [
#     'Please use chained reasoning to put the answer in \\boxed{}.',
#     'Please reflect and verify while reasoning and put the answer in \\boxed{}.',
#     'Solve the following problem using concise and clear reasoning by placing the answer in \\boxed{}.',
#     'Restate the problem, reason step by step, finally put the answer in \\boxed{}.',
#     'Please reason carefully, with the help of a python programme, and put the answer in \\boxed{}.'
#     'You are a helpful and reflective math assistant, please reason step by step to put the answer in \\boxed{}.',
#     # 'As a great math expert, please use concise and clear reasoning to put the answer in \\boxed{}.'
# ]

In [6]:
#thoughts = [
#    'Modular decomposition: Split into 3+ independent lemmas → Synthesize in \\boxed{}',
#    'Extremal principle: Examine boundary cases to constrain solution space → \\boxed{}',
#    'Invariant tracking: Identify conserved quantities through transformations → \\boxed{}',
#    'Dual perspective: Simultaneously develop algebraic & geometric interpretations → \\boxed{}', 
#    'Controlled generalization: Extend specific cases using symmetry groups → \\boxed{}',
#    'Progressive formalization: Begin with intuitive sketches → Incrementally rigorize → \\boxed{}', 
#    'Delayed commitment: Maintain multiple hypothesis paths until step 5 → \\boxed{}',
#    'Ghost parameter technique: Introduce then eliminate auxiliary variables → \\boxed{}',
#    'Topological attack: Reinterpret discrete problems via continuity principles → \\boxed{}'
#]

## Prompt Engineering
### Prompts I added to test whether a different prompt approach would improved DeepSeek Qwen's answers. 
### Both the thoughts in cell above at 32768 max token length with 6 thoughts and mine with 2 thoughts at 8192 max token length resulted in 6/10 on reference set.

In [7]:
thoughts = [
    'You are deeply analytical and detail oriented in your approach. You know the algebraic rules of thumbs to reduce problems quickly. You understand the complete problem statement and solve the problems with deep reasoning. You are careful and proceed with solid logical steps. You answer each solution when you are confident in the solution. You think logically step-by-step utilizing combinations, series, and geometric proofs as necessary. Please . Return final answer within → \\boxed{}, after taking modulo 1000.',
    'You make sure you understand all the given conditions and what the question is asking for. You analyze the problem to determine which areas of mathematics are likely involved such as geometry, number theory, algebra, or combinatorics. You substitute small numbers and specific situaitons to gain intuition and explore potential solutions. You visualize when possible for potential relationships. Once you have a solution, you carefully review your steps to ensure logical reasoning. Return final answer within \\boxed{} after taking modulo 1000'
]

In [8]:
import re

#create single prompt
def make_next_prompt(text,round_idx):
    default_prompt = thoughts[(round_idx+1)%len(thoughts)]
    default_python_code = f"print('{default_prompt}')"
    return default_python_code

#extract python code from response
def extract_python_code(text):
    pattern = r'```python\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)
    if matches:
        ans = "\n\n".join(matches)
        #print(f'Extracted python code: {ans}')
        return ans
    return ""

#extract all code segments
def extract_python_code_list(text):
    pattern = r'```python\s*(.*?)\s*```'
    ans=[]
    matches = re.findall(pattern, text, re.DOTALL)
    for m in matches:
        ans.append(m)
    return ans

#process the code
def process_python_code(query):
    query = "import math\nimport numpy as np\nimport sympy as sp\n" + query
    current_rows = query.strip().split("\n")
    new_rows = []
    for row in current_rows:
        new_rows.append(row)
    ans = "\n".join(new_rows)
    print(f'Processed python code: {ans}')
    return ans

import re

#extract the answer from the boxes
def extract_boxed_texts(text):
    pattern = r'oxed{(.*?)}'
    matches = re.findall(pattern, text)
    if not matches:
        return []
    ans = []
    for content in matches:
        if content.isdigit():
            num = int(content)
        else:
            nums = re.findall(r'\d+', content)
            if not nums:
                continue
            num = int(nums[-1])
        ans.append(num % 1000)
    return ans
    
#extract the integer answer modulo 1000 from the boxes
def extract_boxed_text(text):
    pattern = r'oxed{(.*?)}'
    matches = re.findall(pattern, text)
    if not matches:
        return -1
    content = matches[0]
    if content.isdigit():
        num = int(content)
    else:
        nums = re.findall(r'\d+', content)
        if not nums:
            return -1
        num = int(nums[-1])
    return num % 1000

#select the final answer based on the frequency/ majoity voting
from collections import Counter
def select_answer(answers):
    valid_answers = []
    for answer in answers:
        try:
            if int(answer) == float(answer):
                if 1 < int(answer) < 999 and int(answer) % 100 > 0:
                    valid_answers.append(int(answer))
        except:
            pass
    if not valid_answers:
        return 49
    _, answer = sorted([(v,k) for k,v in Counter(valid_answers).items()], reverse=True)[0]
    return answer%1000

In [9]:
import os
import tempfile
import subprocess

#Python REPL to execute code. taken from NuminaMath Solution
class PythonREPL:
    def __init__(self, timeout=8):
        self.timeout = timeout

    def __call__(self, query):
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_file_path = os.path.join(temp_dir, "tmp.py")
            with open(temp_file_path, "w", encoding="utf-8") as f:
                f.write(query)
            
            try:
                result = subprocess.run(
                    ["python3", temp_file_path],
                    capture_output=True,
                    check=False,
                    text=True,
                    timeout=self.timeout,
                )
            except subprocess.TimeoutExpired:
                return False, f"Execution timed out after {self.timeout} seconds."

            stdout = result.stdout.strip()
            stderr = result.stderr.strip()

            if result.returncode == 0:
                return True, stdout
            else:
                # Process the error message to remove the temporary file path
                # This makes the error message cleaner and more user-friendly
                error_lines = stderr.split("\n")
                cleaned_errors = []
                for line in error_lines:
                    if temp_file_path in line:
                        # Remove the path from the error line
                        line = line.replace(temp_file_path, "<temporary_file>")
                    cleaned_errors.append(line)
                cleaned_error_msg = "\n".join(cleaned_errors)
                # Include stdout in the error case
                combined_output = f"{stdout}\n{cleaned_error_msg}" if stdout else cleaned_error_msg
                return False, combined_output

In [10]:
#sanity check
list_of_texts = [
    tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True
    )
    for messages in [[{"role": "user", "content": "hi"}]]
]

In [11]:
#define the sampling parameters
sampling_params = SamplingParams(
    temperature=0.6,
    
    min_p=0.1,
    skip_special_tokens=True,
    max_tokens=8192,
    # stop=["```output"],
)

#generate prompts in batch
def batch_message_generate(list_of_messages) -> list[list[dict]]:
    list_of_texts = [
        tokenizer.apply_chat_template(
            conversation=messages,
            tokenize=False,
            add_generation_prompt=True
        )
        for messages in list_of_messages
    ]
    
    request_output = llm.generate(
        prompts=list_of_texts,
        sampling_params=sampling_params,
    )
    
    for messages, single_request_output in zip(list_of_messages, request_output):
        messages.append({'role': 'assistant', 'content': single_request_output.outputs[0].text})
        print(messages[-1])

    return list_of_messages

In [12]:
#filter answers from the responses

def batch_message_filter(list_of_messages,list_of_idx) -> tuple[list[list[dict]], list[str]]:
    global answer_contributions
    extracted_answers = []
    list_of_messages_to_keep = []
    list_of_idx_to_keep = []
    for idx,messages in zip(list_of_idx,list_of_messages):
        answers = extract_boxed_texts(messages[-1]['content'])
        if answers:
            extracted_answers.extend(answers)
            for answer in answers:
                answer_contributions[answer].append(idx)
        else:
            list_of_messages_to_keep.append(messages)
            list_of_idx_to_keep.append(idx)
    return list_of_messages_to_keep, extracted_answers, list_of_idx_to_keep

In [13]:

#execute the codes in the responses
def batch_message_execute(list_of_messages,round_idx) -> list[list[dict]]:
    for messages in list_of_messages:
        python_code = extract_python_code(messages[-1]['content'],round_idx)
        python_code = process_python_code(python_code)
        try:
            success, output = PythonREPL()(python_code)
        except Exception as e:
            output = str(e)
        messages.append({'role': 'user', 'content': output})
        print(messages[-1])
    return list_of_messages

#execute the code and generate the answer from responses
def batch_message_execute_and_get_answer(list_of_messages,round_idx) -> tuple[list[list[dict]], list[int]]:
    ans = []
    for messages in list_of_messages:
        python_code = extract_python_code(messages[-1]['content'])
        python_code = process_python_code(python_code)
        try:
            success, output = PythonREPL()(python_code)
            if success:
                patten = r'(\d+)'
                matches = re.findall(patten, output)
                if matches:
                    for match in matches:
                        ans.append(int(match)%1000)
                        ans.append(int(match)%1000) #代码权重高于自然语言，所以添加两次 
        except Exception as e:
            output = str(e)
        print(f'python code output: {output}')
    return ans

#execute code and generate answer for all elements in batch
def batch_message_list_execute_and_get_answer(list_of_messages,round_idx) -> tuple[list[list[dict]], list[int]]:
    ans = []
    for messages in list_of_messages:
        python_code_list = extract_python_code_list(messages[-1]['content'])
        for python_code in python_code_list:
            python_code = process_python_code(python_code)
            try:
                success, output = PythonREPL()(python_code)
                if success:
                    patten = r'(\d+)'
                    matches = re.findall(patten, output)
                    if matches:
                        for match in matches:
                            ans.append(int(match)%1000)
                            ans.append(int(match)%1000) 
            except Exception as e:
                output = str(e)
            print(f'python code output: {output}')
    return ans

In [14]:
import os

import pandas as pd
import polars as pl

#API for competition submission
import kaggle_evaluation.aimo_2_inference_server

In [15]:
#correct answers for reference problems
def get_correct_answer(question):
    if 'Three airline' in question: return 79
    if 'Fred and George' in question: return 250
    if 'Triangle $ABC$' in question: return 180
    if 'Find the three' in question: return 143
    if 'We call a' in question: return 3
    if 'Let $ABC$ be' in question: return 751
    if 'For a positive' in question: return 891
    if 'For positive integers' in question: return 810
    if 'The Fibonacci numbers' in question: return 201
    if 'Alice writes all' in question: return 902
    return 0

# Inference

In [16]:
#predict function to solve single problem

from collections import Counter, defaultdict
g_score = 0
g_count = 0
prompt_score = Counter()
answer_contributions = defaultdict(list)
len_p = len(thoughts)
def predict_for_question(question: str) -> int:
    global g_score
    global g_count
    global prompt_score
    global answer_contributions
    question += "\nIf the final answer is a number larger than 1000, take modulo 1000. "
    if time.time() > cutoff_time: 
        return 210
    print(question)
    
    list_of_messages = [
        [
            {"role": "system", "content": thoughts[k]},
            {"role": "user", "content": question}
        ] for k in range(len_p)
    ]

    all_extracted_answers = []
    list_of_idx = list(range(len(list_of_messages)))
    max_round = 1
    for round_idx in range(max_round):
        print(f"round {round_idx+1}")
        list_of_messages = batch_message_generate(list_of_messages)
        #extracted_python_answer = batch_message_execute_and_get_answer(list_of_messages,round_idx)
        extracted_python_answer = batch_message_list_execute_and_get_answer(list_of_messages,round_idx)
        list_of_messages, extracted_answers, list_of_idx  = batch_message_filter(list_of_messages, list_of_idx)
        all_extracted_answers.extend(extracted_python_answer)
        all_extracted_answers.extend(extracted_answers)
        print("extracted boxed answers:",extracted_answers)
        print("extracted python answers:",extracted_python_answer)
        print("all extracted answers:",all_extracted_answers)
        if not list_of_messages:
            break
        #list_of_messages = batch_message_execute(list_of_messages,round_idx)
    answer = select_answer(all_extracted_answers)
    print("answer:",answer)
    correct_answer = get_correct_answer(question)
    print("correct answer:",correct_answer)
    g_count += 1
    if str(answer) == str(correct_answer):
        g_score += 1

    print(f"score: {g_score}/{g_count}")
    print("\n\n")
    return answer

In [17]:
# Replace this function with your inference code.
# The function should return a single integer between 0 and 999, inclusive.
# Each prediction (except the very first) must be returned within 30 minutes of the question being provided.
def predict(id_: pl.DataFrame, question: pl.DataFrame) -> pl.DataFrame | pd.DataFrame:
    id_ = id_.item(0)
    print("------")
    print(id_)
    
    question = question.item(0)
    answer = predict_for_question(question)
    print(question)
    print("------\n\n\n")
    return pl.DataFrame({'id': id_, 'answer': answer})

In [18]:
pd.read_csv(
    '/kaggle/input/ai-mathematical-olympiad-progress-prize-2/reference.csv'
).drop('answer', axis=1).to_csv('reference.csv', index=False)

In [19]:
inference_server = kaggle_evaluation.aimo_2_inference_server.AIMO2InferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            'reference.csv',
            # "/kaggle/input/ai-mathematical-olympiad-progress-prize-2/test.csv",
        )
    )

------
057f8a
Three airline companies operate flights from Dodola island. Each company has a different schedule of departures. The first company departs every 100 days, the second every 120 days and the third every 150 days. What is the greatest positive integer $d$ for which it is true that there will be $d$ consecutive days without a flight from Dodola island, regardless of the departure times of the various airlines?
If the final answer is a number larger than 1000, take modulo 1000. 
round 1


Processed prompts: 100%|██████████| 2/2 [03:47<00:00, 113.89s/it, est. speed input: 1.91 toks/s, output: 70.03 toks/s]


{'role': 'assistant', 'content': '<think>\nAlright, so I have this problem about three airline companies departing from Dodola island. Each company has its own schedule: the first departs every 100 days, the second every 120 days, and the third every 150 days. I need to find the greatest positive integer \\( d \\) such that there will be \\( d \\) consecutive days without any flights, regardless of when each company starts their departure schedule.\n\nHmm, okay. So, first, I should probably think about the concept of the problem. It seems related to the least common multiple (LCM) because the LCM of their departure intervals would give the period after which their schedules repeat. But the question isn\'t directly asking about when they all depart together again; instead, it\'s about the maximum number of consecutive days without any flights.\n\nWait, so if I can find the maximum gap between two consecutive days where at least one flight departs, then that gap minus one would be the ma

Processed prompts: 100%|██████████| 2/2 [03:46<00:00, 113.38s/it, est. speed input: 1.64 toks/s, output: 67.56 toks/s]


{'role': 'assistant', 'content': "<think>\nOkay, so I need to compute S(S(1) + S(2) + ... + S(N)) where N is 10^100 - 2. Hmm, that's a really big number. Let me break this down step by step.\n\nFirst, let me understand what S(n) is. S(n) is the sum of the digits of n in base 10. For example, S(123) would be 1 + 2 + 3 = 6. So, I need to find the sum of S(1) + S(2) + ... + S(N), and then take the sum of the digits of that total.\n\nGiven that N is 10^100 - 2, which is a 100-digit number where all digits except the first are 9s, right? Because 10^100 is a 1 followed by 100 zeros, so subtracting 2 would give 999...998, with 99 nines and an 8 at the end.\n\nSo, I need to compute the sum of digits from 1 up to 999...998 (with 100 digits). That's a huge range, but maybe there's a pattern or formula I can use.\n\nI remember that the sum of digits from 1 to N can be calculated using some digit-by-digit analysis. Maybe I can consider each digit position separately and compute the total contribut

...

{'role': 'assistant', 'content': '<think>\nAlright, let me try to figure out this problem step by step. So, Alice writes all positive integers from 1 to n on the board, where n is at least 11. Then Bob erases ten of them. The mean of the remaining numbers is given as 3000/37. We need to find the sum S of the numbers Bob erased and then compute the remainder when n times S is divided by 997.\n\nFirst, let\'s parse the problem. The numbers are from 1 to n, so the total number of numbers is n. Bob erases ten of them, so the remaining numbers are n - 10. The mean of these remaining numbers is 3000/37. That means the sum of the remaining numbers is (n - 10) multiplied by (3000/37). \n\nLet me write down what I know:\n\n- Total numbers: n\n- Numbers erased: 10\n- Numbers remaining: n - 10\n- Mean of remaining numbers: 3000/37\n- Sum of remaining numbers: (n - 10) * (3000/37)\n- Sum of all numbers from 1 to n: S_total = n(n + 1)/2\n- Sum of erased numbers: S = S_total - sum of remaining numbe

Processed prompts: 100%|██████████| 2/2 [02:12<00:00, 66.20s/it, est. speed input: 3.81 toks/s, output: 70.15 toks/s]


{'role': 'assistant', 'content': "<think>\nOkay, so I have this problem about Fibonacci numbers and some modular arithmetic. Let me try to break it down step by step. \n\nFirst, the problem is asking about positive integers n strictly less than 10^101. So, n can be any number from 1 up to 10^101 - 1. That's a huge number, so I can't just check each one individually. I need some patterns or properties to figure this out.\n\nThe main conditions are:\n1. n² + (n+1)² is a multiple of 5.\n2. F_{n-1}² + F_n² is not a multiple of 5.\n\nSo, I need to find all n such that the first condition holds, but the second doesn't. Then, count how many such n there are (which is N), and find the number of prime factors of N, counting multiplicity.\n\nAlright, let's tackle each condition one by one.\n\nStarting with the first condition: n² + (n+1)² is divisible by 5.\n\nLet me compute n² + (n+1)² modulo 5. Maybe that will help.\n\nn² + (n+1)² = n² + (n² + 2n + 1) = 2n² + 2n + 1.\n\nSo, 2n² + 2n + 1 ≡ 0 mo

Processed prompts: 100%|██████████| 2/2 [02:22<00:00, 71.15s/it, est. speed input: 2.56 toks/s, output: 70.32 toks/s]

{'role': 'assistant', 'content': '<think>\nOkay, so I need to find a three-digit number \\( n \\) such that if I take any other three-digit number and write it \\( 10^{2024} \\) times in a row, the resulting number is divisible by \\( n \\). Similarly, if I write the same three-digit number \\( 10^{2024} + 2 \\) times in a row, that resulting number should also be divisible by \\( n \\). Hmm, that sounds a bit abstract, but let me try to break it down.\n\nFirst, let me understand what it means to write a number multiple times in a row. For example, if I have a three-digit number, say 123, and I write it twice in a row, I get 123123. If I write it three times, I get 123123123, and so on. So, in general, if I write a number \\( k \\) times in a row, the resulting number is \\( k \\) concatenated copies of the original number.\n\nNow, in mathematical terms, concatenating a number \\( m \\) times can be represented as:\n\n\\[\nN = d \\times 10^{3(m-1)} + d \\times 10^{3(m-2)} + \\dots + d 